In [1]:
# Imports
import nltk.corpus
import nltk.tokenize.punkt
import nltk.stem.snowball
import string
import numpy as np
import pandas as pd
from nltk import pos_tag
from nltk.corpus import wordnet as wn

In [2]:
BL= pd.read_excel('BugList.xls', header=None )
SL= pd.read_excel('StackOverflow.xls', header=None )
#bug_list=np.array(BL)
#so_list=np.array(SL)


In [3]:
bug_list_ids=np.array(BL.iloc[:, :2])
bug_list_des=np.array(BL.iloc[:,2])
so_list_ids=np.array(SL.iloc[:, :3])
so_list_des=np.array(SL.iloc[:,3])

In [4]:
# Semantic  analysis of sentence
def penn_to_wn(tag):
    # Converting a penn Treebank tag to a simplified Wordnet tag "
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('R'):
        return 'r'
    if tag.startswith('V'):
        return 'v'
    if tag.startswith('J'):
        return 'a'
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
    
def find_similarity(token_list1, token_list2):
    #compute the sentence similarity using Wordnet
    # Tokenize and tag
    sent1 = pos_tag(token_list1)
    sent2 = pos_tag(token_list2)
    # Get the synonym set for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sent1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sent2]
    # Filtering out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    # intialise the count
    #print(synsets1)
    #print(synsets2)
    score1, count1 = 0.0, 0
    score2, count2 = 0.0, 0
    # For each word in the first sentence
    for synset in synsets1:
        # finding the similarity value of the most similar word in the other sentence
        best_score=0.0
        for ss in synsets2 :
            if synset.path_similarity(ss) is not None :
                best_score=max(best_score,synset.path_similarity(ss))
        # Check that the similarity could have been computed
        if best_score is not None:
            score1 += best_score
            count1 += 1
            #print(best_score)
            
   
    
    #for each word in the second sentance
    #for synset_2 in synsets2:
        # Get the similarity value of the most similar word in the other sentence
        #best_score2 = max([synset_2.path_similarity(ss1) for ss1 in synsets1])
        # Check that the similarity could have been computed
        #if best_score2 is not None:
        #    score2 += best_score2
        #    count2 += 1
    # Average of avg values of two comparision
    #score = ((score1/count1) + (score2/count2))/2
    if count1 == 0 :
        return 0
    else: 
        score= score1/count1
        return score

In [5]:
def init_list_of_object(size):
    list_object=list()
    for i in range(0,size):
        list_object.append(list())
    return list_object

def format_data(data):
    #tokanize_list=init_list_of_object(size)
    tokanize_lists=[]
    # Getting default English stopwords and extending with punctuation
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(string.punctuation)
    stopwords.append('')
    # Create tokenizer and stemmer
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    for item in data:
        tokens_item = [token.lower().strip(string.punctuation) for token in nltk.word_tokenize(item) \
                    if token.lower().strip(string.punctuation) not in stopwords]
        #stemming the extracted tokens
        #stems_item = [stemmer.stem(token) for token in tokens_item]
        # append the stem_item in tokanise list
        #tokanize_lists.append(stems_item)
        tokanize_lists.append(tokens_item)
    return tokanize_lists

def generate_mapping(bug,so):
    matrix=[]
    for item1 in so:
        so_map_vector_T=[find_similarity(item1,item2) for item2 in bug]
        #so_map_vector_T=[len(set(item).intersection(item2)) / float(len(set(item).union(item2))) for item2 in bug]
        #append the vector int matrix
        matrix.append(so_map_vector_T)
    return matrix
        


In [13]:
def rem_neg_vote_post(so_list_ids,so_list_des):
    for item in range(0,len(so_list_ids)):
        if(so_list_ids[item][2]<0):
            print(so_list_des[item])
            so_list_ids=np.delete(so_list_ids,item,0)
            so_list_des=np.delete(so_list_des,item,0)
            item=item-1
    return so_list_ids,so_list_des   


In [6]:
#main function for all the task
#so_list_ids,so_list_des = rem_neg_vote_post(so_list_ids,so_list_des)
bug_list_des_token=format_data(bug_list_des)
so_list_des_token=format_data(so_list_des)


In [7]:
#print(so_list_des_token)

In [8]:
result= generate_mapping(bug_list_des_token,so_list_des_token)
print(result)

#print(len(a),len(b))
#print(so_list_ids[5][2])

[[0.10833333333333334, 0.10443376068376069, 0.21554487179487178, 0.10833333333333334, 0.14154040404040405, 0.09405964405964407, 0.11837606837606836, 0.05555555555555555, 0.11291486291486291, 0.10202991452991454], [0.15555555555555556, 0.15157509157509158, 0.15157509157509158, 0.15555555555555556, 0.1933333333333333, 0.12631996658312444, 0.16634920634920633, 0.10666666666666666, 0.16037480063795853, 0.1486904761904762], [0.1989795918367347, 0.20622710622710622, 0.15860805860805863, 0.16326530612244897, 0.15146520146520145, 0.13725520511234798, 0.17772108843537412, 0.10476190476190476, 0.17041847041847039, 0.14587257980115123], [0.11282051282051281, 0.17353479853479856, 0.10157712657712659, 0.1001942501942502, 0.11639194139194138, 0.09537037037037037, 0.12209595959595959, 0.041666666666666664, 0.10589225589225588, 0.08700396825396824], [0.11548821548821549, 0.11041666666666666, 0.1173611111111111, 0.11548821548821549, 0.11468253968253966, 0.10634920634920636, 0.1292226292226292, 0.055555

NameError: name 'a' is not defined

In [83]:
def rem_neg_vote_post_list_ll(SL):
    for item in range(0,len(SL)):
        print(SL[item])
        if(SL[item][3] < 0):
            print(SL[item])
            SL.remove(SL[item])        
    return SL 

    for synset in synsets1:
        for ss in synsets2:
            print(synset.path_similarity(ss))



In [ ]:
#ab=rem_neg_vote_post_list(SL)
#print(len(SL))